In [ ]:
from langchain.tools import tool
import os
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [ ]:
db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [ ]:
@tool
def sql_query(query: str) -> str:
    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"
    

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

In [ ]:
agent = create_agent(
    model=llm,
    tools=[sql_query]
)

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [ ]:
print(response["messages"][-1].content)

In [ ]:
from pprint import pprint

pprint(response['messages'])

In [ ]:
print(response["messages"][-3].tool_calls[0]['args']['query'])